<a href="https://colab.research.google.com/github/iamr7d/Deep-Learning/blob/main/basics_of_auto_encoders_using_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [ ]:
# Load and normalize the dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape data to add a channel dimension for grayscale
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [ ]:
# Encoder: Compresses the input image to a latent-space representation
input_img = layers.Input(shape=(28, 28, 1))  # Input layer

# Encoding layers
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)  # Reduces dimension by half
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)  # Further reduces dimension by half

# Encoded (compressed) representation
encoded = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(encoded)  # Final compression step


In [ ]:
from tensorflow.keras.layers import Cropping2D

# Decoder: Reconstructs the image from the encoded representation
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

# Crop the extra pixels to match the input shape (28, 28, 1)
x = Cropping2D(((2, 2), (2, 2)))(x)

# Output layer: Same dimensions as input with a sigmoid activation for pixel values between 0 and 1
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


In [ ]:
# Full autoencoder model
autoencoder = models.Model(input_img, decoded)
autoencoder.summary()

In [ ]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
history = autoencoder.fit(
    x_train, x_train,
    epochs=20,
    batch_size=128,
    shuffle=True,
    validation_data=(x_test, x_test)
)

In [ ]:
import matplotlib.pyplot as plt

# Encode and decode some images from the test set
decoded_imgs = autoencoder.predict(x_test)

# Plot original and reconstructed images for comparison
n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis('off')

    # Reconstructed images
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Reconstructed")
    plt.axis('off')

plt.show()
